In [2]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

### Part 1 - Data Preprocessing


In [3]:
# Importing the dataset
df = pd.read_csv('waterQuality1.csv')
df.head()

,aluminium,ammonia,arsenic,barium,cadmium,chloramine,chromium,copper,flouride,bacteria,...,lead,nitrates,nitrites,mercury,perchlorate,radium,selenium,silver,uranium,is_safe
0,1.65,9.08,0.04,2.85,0.007,0.35,0.83,0.17,0.05,0.20,...,0.054,16.08,1.13,0.007,37.75,6.78,0.08,0.34,0.02,1
1,2.32,21.16,0.01,3.31,0.002,5.28,0.68,0.66,0.90,0.65,...,0.100,2.01,1.93,0.003,32.26,3.21,0.08,0.27,0.05,1
2,1.01,14.02,0.04,0.58,0.008,4.24,0.53,0.02,0.99,0.05,...,0.078,14.16,1.11,0.006,50.28,7.07,0.07,0.44,0.01,0
3,1.36,11.33,0.04,2.96,0.001,7.23,0.03,1.66,1.08,0.71,...,0.016,1.41,1.29,0.004,9.12,1.72,0.02,0.45,0.05,1
4,0.92,24.33,0.03,0.20,0.006,2.67,0.69,0.57,0.61,0.13,...,0.117,6.74,1.11,0.003,16.90,2.41,0.02,0.06,0.02,1


In [4]:
df.shape

(7999, 21)

In [5]:
df.isin([np.inf, -np.inf]).sum()

aluminium      0
ammonia        0
arsenic        0
barium         0
cadmium        0
chloramine     0
chromium       0
copper         0
flouride       0
bacteria       0
viruses        0
lead           0
nitrates       0
nitrites       0
mercury        0
perchlorate    0
radium         0
selenium       0
silver         0
uranium        0
is_safe        0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7999 entries, 0 to 7998
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   aluminium    7999 non-null   float64
 1   ammonia      7999 non-null   object 
 2   arsenic      7999 non-null   float64
 3   barium       7999 non-null   float64
 4   cadmium      7999 non-null   float64
 5   chloramine   7999 non-null   float64
 6   chromium     7999 non-null   float64
 7   copper       7999 non-null   float64
 8   flouride     7999 non-null   float64
 9   bacteria     7999 non-null   float64
 10  viruses      7999 non-null   float64
 11  lead         7999 non-null   float64
 12  nitrates     7999 non-null   float64
 13  nitrites     7999 non-null   float64
 14  mercury      7999 non-null   float64
 15  perchlorate  7999 non-null   float64
 16  radium       7999 non-null   float64
 17  selenium     7999 non-null   float64
 18  silver       7999 non-null   float64
 19  uraniu

In [7]:
df.isnull().sum()

aluminium      0
ammonia        0
arsenic        0
barium         0
cadmium        0
chloramine     0
chromium       0
copper         0
flouride       0
bacteria       0
viruses        0
lead           0
nitrates       0
nitrites       0
mercury        0
perchlorate    0
radium         0
selenium       0
silver         0
uranium        0
is_safe        0
dtype: int64

convert dbject datatype to float

In [8]:
for i in range(df.shape[1]):
    df.iloc[:,i] =  pd.to_numeric(df.iloc[:,i], errors='coerce')

In [9]:
df["is_safe"] = pd.to_numeric(df["is_safe"], errors='coerce')

In [10]:
df["is_safe"] = df["is_safe"]== 1.0

In [11]:
df["is_safe"]

0        True
1        True
2       False
3        True
4        True
        ...  
7994     True
7995     True
7996     True
7997     True
7998     True
Name: is_safe, Length: 7999, dtype: bool

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7999 entries, 0 to 7998
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   aluminium    7999 non-null   float64
 1   ammonia      7996 non-null   float64
 2   arsenic      7999 non-null   float64
 3   barium       7999 non-null   float64
 4   cadmium      7999 non-null   float64
 5   chloramine   7999 non-null   float64
 6   chromium     7999 non-null   float64
 7   copper       7999 non-null   float64
 8   flouride     7999 non-null   float64
 9   bacteria     7999 non-null   float64
 10  viruses      7999 non-null   float64
 11  lead         7999 non-null   float64
 12  nitrates     7999 non-null   float64
 13  nitrites     7999 non-null   float64
 14  mercury      7999 non-null   float64
 15  perchlorate  7999 non-null   float64
 16  radium       7999 non-null   float64
 17  selenium     7999 non-null   float64
 18  silver       7999 non-null   float64
 19  uraniu

In [13]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [14]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [15]:
X = np.nan_to_num(X) 

In [21]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)


In [ ]:
def build_clf(hiddenLayersNum,Neurons,Opt,activationFunctions):
# creating the layers of the NN
   ann = Sequential()
    for i in range(hiddenLayers):
        ann.add(Dense(units=neurons, activation=activationFunctions))
    ann.compile(optimizer = Opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return ann

#model=KerasClassifier(build_fn=build_clf, verbose=1)
model=KerasClassifier(build_fn=build_clf,epochs=100, batch_size=10, verbose=1)
params={'batch_size':[100, 30],
        'nb_epoch':[10,20],
        'unit':[5,6,],
        #'Opt':['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
        "hiddenLayers" = [2,3]
		}
gs=GridSearchCV(estimator=model, param_grid=params, cv=3)
# now fit the dataset to the GridSearchCV object.
gs = gs.fit(X_train, y_train)

result = pd.DataFrame(gs.cv_results_)[sorted(gs.cv_results_.keys())]

best_params=gs.best_params_
accuracy=gs.best_score_

print("Best Accuracy: {:.2f} %".format(accuracy*100))
print("Best Parameters:", best_params)

In [25]:
def build_clf(unit,Opt,act1,act2):
# creating the layers of the NN
    ann =Sequential()
    ann.add(Dense(units=unit, activation=act1))
    ann.add(Dense(units=unit, activation=act2))
    ann.add(Dense(units=1, activation='sigmoid'))
    ann.compile(optimizer = Opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return ann

#model=KerasClassifier(build_fn=build_clf, verbose=1)
model=KerasClassifier(build_fn=build_clf,epochs=10, batch_size=10, verbose=1)
params={#'batch_size':[100, 20, 50, 25, 32],
    #'nb_epoch':[200, 100, 300, 400],
    'unit':[5,6, 10, 11, 12, 15],
    'Opt':['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
    'act1':["linear","relu","sigmoid"],
    'act2':["linear","relu","sigmoid"]}
gs=GridSearchCV(estimator=model, param_grid=params, cv=2)
# now fit the dataset to the GridSearchCV object.
gs = gs.fit(X_train, y_train)

result = pd.DataFrame(gs.cv_results_)[sorted(gs.cv_results_.keys())]

best_params=gs.best_params_
accuracy=gs.best_score_

print("Best Accuracy: {:.2f} %".format(accuracy*100))
print("Best Parameters:", best_params)

Epoch 1/10


C:\Users\Rahomy\AppData\Local\Temp/ipykernel_7532/2436515640.py:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=build_clf,epochs=10, batch_size=10, verbose=1)


320/320 [==============================] - 1s 3ms/step - loss: 0.4729 - accuracy: 0.8128
Epoch 2/10
320/320 [==============================] - 1s 3ms/step - loss: 0.3394 - accuracy: 0.8837
Epoch 3/10
320/320 [==============================] - 1s 4ms/step - loss: 0.3083 - accuracy: 0.8893
Epoch 4/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2847 - accuracy: 0.8947
Epoch 5/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2683 - accuracy: 0.9018
Epoch 6/10
320/320 [==============================] - 1s 3ms/step - loss: 0.2582 - accuracy: 0.9062
Epoch 7/10
320/320 [==============================] - 1s 3ms/step - loss: 0.2520 - accuracy: 0.9062
Epoch 8/10
320/320 [==============================] - 1s 4ms/step - loss: 0.2485 - accuracy: 0.9081
Epoch 9/10
320/320 [==============================] - 1s 4ms/step - loss: 0.2465 - accuracy: 0.9100
Epoch 10/10
320/320 [==============================] - 1s 1ms/step - loss: 0.2474 - accuracy: 0.9109
Epoch 1/10

320/320 [==============================] - 1s 2ms/step - loss: 0.2456 - accuracy: 0.9072
Epoch 7/10
320/320 [==============================] - 1s 2ms/step - loss: 0.2449 - accuracy: 0.9094
Epoch 8/10
320/320 [==============================] - 1s 2ms/step - loss: 0.2446 - accuracy: 0.9103
Epoch 9/10
320/320 [==============================] - 1s 2ms/step - loss: 0.2447 - accuracy: 0.9075
Epoch 10/10
320/320 [==============================] - 1s 2ms/step - loss: 0.2459 - accuracy: 0.9084
Epoch 1/10
320/320 [==============================] - 1s 2ms/step - loss: 0.4674 - accuracy: 0.7934
Epoch 2/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2989 - accuracy: 0.8872
Epoch 3/10
320/320 [==============================] - 1s 2ms/step - loss: 0.2692 - accuracy: 0.8997
Epoch 4/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2566 - accuracy: 0.9031
Epoch 5/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2500 - accuracy: 0.9081
Epoch 6/10

320/320 [==============================] - 1s 3ms/step - loss: 0.2338 - accuracy: 0.9100
Epoch 1/10
320/320 [==============================] - 1s 1ms/step - loss: 0.4456 - accuracy: 0.8328
Epoch 2/10
320/320 [==============================] - 1s 3ms/step - loss: 0.3396 - accuracy: 0.8869
Epoch 3/10
320/320 [==============================] - 1s 3ms/step - loss: 0.3132 - accuracy: 0.8869
Epoch 4/10
320/320 [==============================] - 1s 3ms/step - loss: 0.2952 - accuracy: 0.8869
Epoch 5/10
320/320 [==============================] - 1s 3ms/step - loss: 0.2830 - accuracy: 0.8869
Epoch 6/10
320/320 [==============================] - 1s 5ms/step - loss: 0.2739 - accuracy: 0.8869
Epoch 7/10
320/320 [==============================] - 1s 3ms/step - loss: 0.2678 - accuracy: 0.8869
Epoch 8/10
320/320 [==============================] - 1s 4ms/step - loss: 0.2638 - accuracy: 0.8872
Epoch 9/10
320/320 [==============================] - 2s 5ms/step - loss: 0.2609 - accuracy: 0.8872
Epoch 10/10

...

320/320 [==============================] - 0s 1ms/step - loss: 0.3183 - accuracy: 0.8869
Epoch 3/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2943 - accuracy: 0.8869
Epoch 4/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2778 - accuracy: 0.8869
Epoch 5/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2644 - accuracy: 0.8878
Epoch 6/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2546 - accuracy: 0.8919
Epoch 7/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2478 - accuracy: 0.9047
Epoch 8/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2437 - accuracy: 0.9081
Epoch 9/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2407 - accuracy: 0.9116
Epoch 10/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2395 - accuracy: 0.9122
Epoch 1/10
320/320 [==============================] - 1s 1ms/step - loss: 0.3540 - accuracy: 0.8825
Epoch 2/10

320/320 [==============================] - 0s 1ms/step - loss: 0.2351 - accuracy: 0.9162
Epoch 8/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2329 - accuracy: 0.9165
Epoch 9/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2303 - accuracy: 0.9162
Epoch 10/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2299 - accuracy: 0.9166
Epoch 1/10
320/320 [==============================] - 1s 1ms/step - loss: 0.3188 - accuracy: 0.8863
Epoch 2/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2624 - accuracy: 0.8925
Epoch 3/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2479 - accuracy: 0.9044
Epoch 4/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2424 - accuracy: 0.9066
Epoch 5/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2394 - accuracy: 0.9094
Epoch 6/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2361 - accuracy: 0.9094
Epoch 7/10

320/320 [==============================] - 1s 1ms/step - loss: 0.4059 - accuracy: 0.8471
Epoch 2/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2917 - accuracy: 0.8825
Epoch 3/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2634 - accuracy: 0.8884
Epoch 4/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2488 - accuracy: 0.9022
Epoch 5/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2411 - accuracy: 0.9087
Epoch 6/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2366 - accuracy: 0.9134
Epoch 7/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2330 - accuracy: 0.9122
Epoch 8/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2283 - accuracy: 0.9147
Epoch 9/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2247 - accuracy: 0.9156
Epoch 10/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2236 - accuracy: 0.9222
Epoch 1/10

320/320 [==============================] - 0s 1ms/step - loss: 0.3026 - accuracy: 0.8869
Epoch 7/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2906 - accuracy: 0.8869
Epoch 8/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2794 - accuracy: 0.8869
Epoch 9/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2696 - accuracy: 0.8869
Epoch 10/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2628 - accuracy: 0.8825
Epoch 1/10
320/320 [==============================] - 1s 1ms/step - loss: 0.3904 - accuracy: 0.8825
Epoch 2/10
320/320 [==============================] - 0s 1ms/step - loss: 0.3419 - accuracy: 0.8825
Epoch 3/10
320/320 [==============================] - 0s 1ms/step - loss: 0.3281 - accuracy: 0.8825
Epoch 4/10
320/320 [==============================] - 0s 1ms/step - loss: 0.3169 - accuracy: 0.8825
Epoch 5/10
320/320 [==============================] - 1s 2ms/step - loss: 0.3057 - accuracy: 0.8825
Epoch 6/10

320/320 [==============================] - 0s 1ms/step - loss: 0.2335 - accuracy: 0.9184
Epoch 1/10
320/320 [==============================] - 1s 1ms/step - loss: 0.4324 - accuracy: 0.8531
Epoch 2/10
320/320 [==============================] - 0s 1ms/step - loss: 0.3200 - accuracy: 0.8869
Epoch 3/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2999 - accuracy: 0.8869
Epoch 4/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2823 - accuracy: 0.8869
Epoch 5/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2672 - accuracy: 0.8869
Epoch 6/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2552 - accuracy: 0.8884
Epoch 7/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2461 - accuracy: 0.9062
Epoch 8/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2389 - accuracy: 0.9141
Epoch 9/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2335 - accuracy: 0.9153
Epoch 10/10

In [16]:
# Fitting SVM woth another kernal to the Training set 
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

SVC(random_state=0)

In [17]:
# Predicting the Test set results
y_pred_SVM_rbf = classifier.predict(X_test)
y_pred_SVM_rbf

array([False, False, False, ..., False,  True, False])

In [242]:
import sklearn.metrics as metrics
accuracy = metrics.accuracy_score(y_test,y_pred_SVM_rbf)
accuracy

0.933125

In [243]:
# Fitting K-NN to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [244]:
y_pred_K_NN = classifier.predict(X_test)
y_pred_K_NN

array([False, False, False, ..., False, False, False])

In [246]:
accuracy = metrics.accuracy_score(y_test,y_pred_K_NN)
accuracy

0.9075

### Part 2 - Building the ANN
 

In [247]:
from sklearn.metrics import confusion_matrix, accuracy_score
def Model(hiddenLayers,neurons,activationFunctions,batchSize,epochs):
    ann = Sequential()
    for i in range(hiddenLayers):
        ann.add(Dense(units=neurons[i], activation=activationFunctions[i]))

    ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    ann.fit(X_train, y_train, validation_data=(X_val, y_val),batch_size = batchSize, epochs = epochs, verbose=1)
    # Predicting the Test set results
    y_pred_train = ann.predict(X_train)
    y_pred_train = (y_pred_train > 0.5)
    y_pred = ann.predict(X_test)
    y_pred = (y_pred > 0.5)
    print(y_test)
    # Making the Confusion Matrix
    train = accuracy_score(y_train, y_pred_train)
    test = accuracy_score(y_test, y_pred)
    return train*100,test*100

## Part 3 - Training the ANN


In [248]:
trainingAccuracy = []
testingAccuracy = []
def Run(numberOfIterations ,hiddenLayers ,neurons,activationFunctions,batchSize,epochs):
    for i in range(numberOfIterations):
        print("iteration: ",i)
        train,test = Model(hiddenLayers[i],neurons[i],activationFunctions[i],batchSize[i],epochs[i])
        print(train,test,"iteratin: ",i)
        trainingAccuracy.append(train)
        testingAccuracy.append(test)
    maxTrain = max(trainingAccuracy)
    indexMaxTrain = trainingAccuracy.index(maxTrain)
    maxTest = max(testingAccuracy)
    indexMaxTest = testingAccuracy.index(maxTest)
    return maxTrain,indexMaxTrain,maxTest,indexMaxTest

Adding the values of the hyperparameters in dictionary
 > **number of hidden layers**
 
 > **number of neurons of each hidden layer**
 
 > **the activation function of each hidden layer**
 
 > **batch size**
 
 > **number of epochs in dictionary**

In [251]:

dic = { 1  : [3, [5,5,1], ["linear","relu","sigmoid"],30, 100],
        2  : [5, [6,6,6,2,1], ["linear","relu","relu","sigmoid","linear"], 32, 150],
        3  : [4, [6,4,5,1], ["linear","relu","relu","sigmoid"], 30, 150],
        4  : [5, [4,4,4,2,1], ["linear","relu","relu","sigmoid","sigmoid"], 32, 100],
        5  : [3, [6,5,1], ["relu","relu","sigmoid"], 30,150]
      }
l = list(dic.values())

numberOfIterations = len(dic)
hiddenLayers = [row[0] for row in l]
neurons = [row[1] for row in l]
activationFunctions = [row[2] for row in l]
batchSize = [row[3] for row in l]
epochs = [row[4] for row in l]

Run(numberOfIterations ,hiddenLayers ,neurons,activationFunctions,batchSize,epochs)

iteration:  0
Epoch 1/100
192/192 [==============================] - 1s 2ms/step - loss: 0.5965 - accuracy: 0.7022 - val_loss: 0.4085 - val_accuracy: 0.8734
Epoch 2/100
192/192 [==============================] - 0s 1ms/step - loss: 0.3542 - accuracy: 0.8837 - val_loss: 0.3165 - val_accuracy: 0.8797
Epoch 3/100
192/192 [==============================] - 0s 2ms/step - loss: 0.3069 - accuracy: 0.8850 - val_loss: 0.2882 - val_accuracy: 0.8813
Epoch 4/100
192/192 [==============================] - 0s 2ms/step - loss: 0.2834 - accuracy: 0.8859 - val_loss: 0.2658 - val_accuracy: 0.8828
Epoch 5/100
192/192 [==============================] - 1s 3ms/step - loss: 0.2602 - accuracy: 0.8934 - val_loss: 0.2421 - val_accuracy: 0.8953
Epoch 6/100
192/192 [==============================] - 1s 3ms/step - loss: 0.2392 - accuracy: 0.9083 - val_loss: 0.2222 - val_accuracy: 0.9125
Epoch 7/100
192/192 [==============================] - 1s 4ms/step - loss: 0.2228 - accuracy: 0.9196 - val_loss: 0.2087 - val_ac

Epoch 58/100
192/192 [==============================] - 0s 2ms/step - loss: 0.1330 - accuracy: 0.9536 - val_loss: 0.1424 - val_accuracy: 0.9484
Epoch 59/100
192/192 [==============================] - 0s 2ms/step - loss: 0.1331 - accuracy: 0.9543 - val_loss: 0.1392 - val_accuracy: 0.9500
Epoch 60/100
192/192 [==============================] - 1s 5ms/step - loss: 0.1320 - accuracy: 0.9526 - val_loss: 0.1398 - val_accuracy: 0.9469
Epoch 61/100
192/192 [==============================] - 1s 3ms/step - loss: 0.1320 - accuracy: 0.9529 - val_loss: 0.1401 - val_accuracy: 0.9500
Epoch 62/100
192/192 [==============================] - 1s 4ms/step - loss: 0.1311 - accuracy: 0.9545 - val_loss: 0.1401 - val_accuracy: 0.9484
Epoch 63/100
192/192 [==============================] - 1s 6ms/step - loss: 0.1310 - accuracy: 0.9535 - val_loss: 0.1405 - val_accuracy: 0.9500
Epoch 64/100
192/192 [==============================] - 0s 2ms/step - loss: 0.1306 - accuracy: 0.9543 - val_loss: 0.1398 - val_accuracy:

Epoch 71/150
180/180 [==============================] - 1s 4ms/step - loss: 0.1305 - accuracy: 0.9522 - val_loss: 0.1576 - val_accuracy: 0.9547
Epoch 72/150
180/180 [==============================] - 0s 3ms/step - loss: 0.1321 - accuracy: 0.9529 - val_loss: 0.1657 - val_accuracy: 0.9516
Epoch 73/150
180/180 [==============================] - 0s 3ms/step - loss: 0.1280 - accuracy: 0.9526 - val_loss: 0.1673 - val_accuracy: 0.9563
Epoch 74/150
180/180 [==============================] - 1s 5ms/step - loss: 0.1257 - accuracy: 0.9533 - val_loss: 0.1651 - val_accuracy: 0.9578
Epoch 75/150
180/180 [==============================] - 1s 4ms/step - loss: 0.1229 - accuracy: 0.9555 - val_loss: 0.1624 - val_accuracy: 0.9563
Epoch 76/150
180/180 [==============================] - 1s 4ms/step - loss: 0.1213 - accuracy: 0.9566 - val_loss: 0.1595 - val_accuracy: 0.9563
Epoch 77/150
180/180 [==============================] - 0s 3ms/step - loss: 0.1404 - accuracy: 0.9521 - val_loss: 0.1400 - val_accuracy:

Epoch 128/150
180/180 [==============================] - 1s 6ms/step - loss: 0.1227 - accuracy: 0.9552 - val_loss: 0.1491 - val_accuracy: 0.9578
Epoch 129/150
180/180 [==============================] - 1s 3ms/step - loss: 0.1437 - accuracy: 0.9505 - val_loss: 0.1865 - val_accuracy: 0.9484
Epoch 130/150
180/180 [==============================] - 1s 4ms/step - loss: 0.1575 - accuracy: 0.9491 - val_loss: 0.1501 - val_accuracy: 0.9531
Epoch 131/150
180/180 [==============================] - 0s 2ms/step - loss: 0.1322 - accuracy: 0.9533 - val_loss: 0.1463 - val_accuracy: 0.9500
Epoch 132/150
180/180 [==============================] - 1s 5ms/step - loss: 0.1262 - accuracy: 0.9562 - val_loss: 0.1443 - val_accuracy: 0.9516
Epoch 133/150
180/180 [==============================] - 1s 4ms/step - loss: 0.1256 - accuracy: 0.9555 - val_loss: 0.1471 - val_accuracy: 0.9484
Epoch 134/150
180/180 [==============================] - 1s 3ms/step - loss: 0.1174 - accuracy: 0.9550 - val_loss: 0.1431 - val_ac

Epoch 91/150
192/192 [==============================] - 1s 5ms/step - loss: 0.1452 - accuracy: 0.9474 - val_loss: 0.1741 - val_accuracy: 0.9375
Epoch 92/150
192/192 [==============================] - 1s 4ms/step - loss: 0.1445 - accuracy: 0.9500 - val_loss: 0.1749 - val_accuracy: 0.9328
Epoch 93/150
192/192 [==============================] - 1s 3ms/step - loss: 0.1447 - accuracy: 0.9479 - val_loss: 0.1767 - val_accuracy: 0.9359
Epoch 94/150
192/192 [==============================] - 1s 3ms/step - loss: 0.1439 - accuracy: 0.9498 - val_loss: 0.1759 - val_accuracy: 0.9359
Epoch 95/150
192/192 [==============================] - 1s 4ms/step - loss: 0.1436 - accuracy: 0.9489 - val_loss: 0.1760 - val_accuracy: 0.9344
Epoch 96/150
192/192 [==============================] - 1s 4ms/step - loss: 0.1430 - accuracy: 0.9479 - val_loss: 0.1767 - val_accuracy: 0.9344
Epoch 97/150
192/192 [==============================] - 1s 3ms/step - loss: 0.1429 - accuracy: 0.9484 - val_loss: 0.1757 - val_accuracy:

180/180 [==============================] - 1s 4ms/step - loss: 0.2193 - accuracy: 0.9305 - val_loss: 0.2227 - val_accuracy: 0.9266
Epoch 54/100
180/180 [==============================] - 1s 5ms/step - loss: 0.2171 - accuracy: 0.9311 - val_loss: 0.2200 - val_accuracy: 0.9281
Epoch 55/100
180/180 [==============================] - 1s 4ms/step - loss: 0.2146 - accuracy: 0.9337 - val_loss: 0.2181 - val_accuracy: 0.9297
Epoch 56/100
180/180 [==============================] - 1s 3ms/step - loss: 0.2112 - accuracy: 0.9371 - val_loss: 0.2120 - val_accuracy: 0.9328
Epoch 57/100
180/180 [==============================] - 1s 6ms/step - loss: 0.2084 - accuracy: 0.9378 - val_loss: 0.2111 - val_accuracy: 0.9312
Epoch 58/100
180/180 [==============================] - 1s 4ms/step - loss: 0.2059 - accuracy: 0.9391 - val_loss: 0.2076 - val_accuracy: 0.9328
Epoch 59/100
180/180 [==============================] - 1s 3ms/step - loss: 0.2031 - accuracy: 0.9406 - val_loss: 0.2049 - val_accuracy: 0.9344
Epoch

Epoch 66/150
192/192 [==============================] - 0s 1ms/step - loss: 0.1090 - accuracy: 0.9613 - val_loss: 0.1469 - val_accuracy: 0.9500
Epoch 67/150
192/192 [==============================] - 0s 2ms/step - loss: 0.1100 - accuracy: 0.9609 - val_loss: 0.1476 - val_accuracy: 0.9531
Epoch 68/150
192/192 [==============================] - 1s 5ms/step - loss: 0.1083 - accuracy: 0.9621 - val_loss: 0.1494 - val_accuracy: 0.9531
Epoch 69/150
192/192 [==============================] - 1s 3ms/step - loss: 0.1089 - accuracy: 0.9613 - val_loss: 0.1455 - val_accuracy: 0.9500
Epoch 70/150
192/192 [==============================] - 0s 2ms/step - loss: 0.1081 - accuracy: 0.9616 - val_loss: 0.1456 - val_accuracy: 0.9469
Epoch 71/150
192/192 [==============================] - 1s 4ms/step - loss: 0.1079 - accuracy: 0.9616 - val_loss: 0.1477 - val_accuracy: 0.9547
Epoch 72/150
192/192 [==============================] - 0s 2ms/step - loss: 0.1081 - accuracy: 0.9621 - val_loss: 0.1451 - val_accuracy:

Epoch 123/150
192/192 [==============================] - 0s 2ms/step - loss: 0.1011 - accuracy: 0.9609 - val_loss: 0.1367 - val_accuracy: 0.9500
Epoch 124/150
192/192 [==============================] - 0s 2ms/step - loss: 0.1003 - accuracy: 0.9618 - val_loss: 0.1391 - val_accuracy: 0.9484
Epoch 125/150
192/192 [==============================] - 1s 4ms/step - loss: 0.1007 - accuracy: 0.9620 - val_loss: 0.1406 - val_accuracy: 0.9453
Epoch 126/150
192/192 [==============================] - 1s 3ms/step - loss: 0.1001 - accuracy: 0.9635 - val_loss: 0.1381 - val_accuracy: 0.9531
Epoch 127/150
192/192 [==============================] - 0s 2ms/step - loss: 0.0997 - accuracy: 0.9630 - val_loss: 0.1376 - val_accuracy: 0.9484
Epoch 128/150
192/192 [==============================] - 1s 3ms/step - loss: 0.1010 - accuracy: 0.9623 - val_loss: 0.1399 - val_accuracy: 0.9547
Epoch 129/150
192/192 [==============================] - 0s 2ms/step - loss: 0.1002 - accuracy: 0.9616 - val_loss: 0.1365 - val_ac

(96.33616947386699, 4, 95.4375, 0)

In [252]:
trainingAccuracy , testingAccuracy

([95.95415870810905,
  95.81524570237889,
  95.3464143080396,
  95.84997395381143,
  96.33616947386699],
 [95.4375, 94.875, 94.125, 94.5625, 95.0])

### To test the code manually

In [253]:
Model(5,[5,5,3,2,1],["linear","relu","relu","sigmoid","linear"],10,5)

Epoch 1/5
576/576 [==============================] - 2s 2ms/step - loss: 0.3803 - accuracy: 0.8850 - val_loss: 0.3352 - val_accuracy: 0.8813
Epoch 2/5
576/576 [==============================] - 1s 2ms/step - loss: 0.2995 - accuracy: 0.8850 - val_loss: 0.3065 - val_accuracy: 0.8813
Epoch 3/5
576/576 [==============================] - 1s 2ms/step - loss: 0.2827 - accuracy: 0.8850 - val_loss: 0.2942 - val_accuracy: 0.8813
Epoch 4/5
576/576 [==============================] - 1s 2ms/step - loss: 0.2656 - accuracy: 0.8849 - val_loss: 0.2680 - val_accuracy: 0.8875
Epoch 5/5
576/576 [==============================] - 1s 2ms/step - loss: 0.2693 - accuracy: 0.8885 - val_loss: 0.2825 - val_accuracy: 0.8969
[False False False ... False  True False]


(89.78989407883313, 89.1875)

In [254]:
Model(3,[6,6,1],["relu","relu","relu"],20,5)

Epoch 1/5
288/288 [==============================] - 1s 2ms/step - loss: 1.5545 - accuracy: 0.8835 - val_loss: 1.2437 - val_accuracy: 0.8813
Epoch 2/5
288/288 [==============================] - 0s 2ms/step - loss: 1.1716 - accuracy: 0.8786 - val_loss: 0.8996 - val_accuracy: 0.8891
Epoch 3/5
288/288 [==============================] - 0s 1ms/step - loss: 0.8237 - accuracy: 0.8705 - val_loss: 0.6801 - val_accuracy: 0.8875
Epoch 4/5
288/288 [==============================] - 0s 2ms/step - loss: 0.6859 - accuracy: 0.8686 - val_loss: 0.6146 - val_accuracy: 0.8797
Epoch 5/5
288/288 [==============================] - 1s 4ms/step - loss: 0.5514 - accuracy: 0.8717 - val_loss: 0.4216 - val_accuracy: 0.8766
[False False False ... False  True False]


(87.04636221566244, 87.125)

### To test the model in direct code

In [40]:
ann = Sequential()

# Adding the input layer and the first hidden layer
ann.add(Dense(units=6, activation='linear'))

# Adding the second hidden layer
ann.add(Dense(units=6, activation='linear'))

# Adding the output layer
ann.add(Dense(units=1, activation='linear'))

# Part 3 - Training the ANN

# Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the ANN on the Training set
ann.fit(X_train, y_train, validation_data=(X_val, y_val),
        batch_size = 25, epochs = 100, verbose=1)

# Model Summary
ann.summary()

# Model Visualization
plot_model(ann, to_file='ANN_Seq.png', show_shapes=True, show_layer_names=True)

# Part 4 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
Acc = accuracy_score(y_test, y_pred)
print("Accuracy:" ,(Acc*100))

# Cross Validation
kfold = KFold(n_splits=5, shuffle=True)
fold_no = 1
acc_per_fold = []
loss_per_fold = []
for train_index, val_index in kfold.split(X_train, y_train):
    ann = Sequential()
    ann.add(Dense(units=6, activation='relu'))
    ann.add(Dense(units=6, activation='relu'))
    ann.add(Dense(units=1, activation='sigmoid'))
    
    ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    ann.fit(X_train[train_index], y_train[train_index], batch_size = 32, epochs = 100, verbose=1)
    
    # Generate generalization metrics
    scores = ann.evaluate(X_train[val_index], y_train[val_index], verbose=1)
    print(f'Score for fold {fold_no}: {ann.metrics_names[0]} of {scores[0]}; {ann.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    # Increase fold number
    fold_no = fold_no + 1

acc_per_fold = np.array(acc_per_fold)
print("Accuracy: {:.2f} %".format(acc_per_fold.mean()))
print("Standard Deviation: {:.2f} %".format(acc_per_fold.std()))


Epoch 1/100
256/256 [==============================] - 2s 6ms/step - loss: 4.2527 - accuracy: 0.6421 - val_loss: 2.9570 - val_accuracy: 0.7250
Epoch 2/100
256/256 [==============================] - 2s 6ms/step - loss: 2.8895 - accuracy: 0.7317 - val_loss: 2.8062 - val_accuracy: 0.7359
Epoch 3/100
256/256 [==============================] - 1s 4ms/step - loss: 2.6527 - accuracy: 0.7493 - val_loss: 2.5706 - val_accuracy: 0.7656
Epoch 4/100
256/256 [==============================] - 2s 6ms/step - loss: 2.3341 - accuracy: 0.7715 - val_loss: 2.1585 - val_accuracy: 0.7906
Epoch 5/100
256/256 [==============================] - 1s 4ms/step - loss: 1.9193 - accuracy: 0.8036 - val_loss: 1.7366 - val_accuracy: 0.8141
Epoch 6/100
256/256 [==============================] - 2s 6ms/step - loss: 1.7331 - accuracy: 0.8217 - val_loss: 1.6135 - val_accuracy: 0.8297
Epoch 7/100
256/256 [==============================] - 0s 2ms/step - loss: 1.6057 - accuracy: 0.8343 - val_loss: 1.4889 - val_accuracy: 0.8406

KeyboardInterrupt: 

### Finnaly I have the accuracy in all differernt hyperparameters
> The Best Accuracy: 95.43%

> The first hyper parameters